### Lableling The images to the word it represesnts

In [ ]:
sentences_txt_path = stow.join('Datasets', 'IAM_Sentences', 'ascii', 'sentences.txt')
sentences_folder_path = stow.join('Datasets', 'IAM_Sentences', 'sentences')

dataset, vocab, max_len = [], set(), 0
words = open(sentences_txt_path, "r").readlines()
for line in tqdm(words):
    if line.startswith("#"):
        continue

    line_split = line.split(" ")
    if line_split[2] == "err":
        continue

    folder1 = line_split[0][:3]
    folder2 = line_split[0][:8]
    file_name = line_split[0] + ".png"
    label = line_split[-1].rstrip('\n')

    # recplace '|' with ' ' in label
    label = label.replace('|', ' ')

    rel_path = stow.join(sentences_folder_path, folder1, folder2, file_name)
    if not stow.exists(rel_path):
        continue

    dataset.append([rel_path, label])
    vocab.update(list(label))
    max_len = max(max_len, len(label))

In [ ]:
# prompt: I have used IAM dataset in which there are images of words written by hands (Handwritten)  but they are not labeled .can i used the prebuilt cnn to recognize it ?

# Pre-trained CNNs are typically trained on object recognition tasks,
# and may not perform well on handwritten text recognition without fine-tuning.

# Install necessary libraries
!pip install tensorflow keras

# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load a pre-trained CNN model (Example: ResNet50)
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Add new classification layers on top of the base model
model = keras.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(number_of_classes, activation='softmax')  # Replace 'number_of_classes' with the number of classes in your dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess your IAM dataset images (resize, normalize, etc.)
# ...

# Train the model on your preprocessed IAM dataset
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Evaluate the model on a test set
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy:', accuracy)

# Use the trained model to recognize handwritten text
predictions = model.predict(new_images)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
def load_dataset():
    # This function should load images and their corresponding labels
    # For example purposes, assume we have image_data and text_labels
    image_data = ...  # Load image data here
    text_labels = ...  # Load corresponding text labels here

    return image_data, text_labels

image_data, text_labels = load_dataset()

# Create a mapping from unique characters to indices
vocab = sorted(set(''.join(text_labels)))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert text labels to integer representation
text_as_int = [[char2idx[c] for c in label] for label in text_labels]

# Pad sequences to the same length
max_length = max(len(label) for label in text_as_int)
text_as_int = pad_sequences(text_as_int, maxlen=max_length, padding='post')

# One-hot encode the labels
text_as_int = [to_categorical(label, num_classes=len(vocab)) for label in text_as_int]

# Convert image data to numpy array
image_data = np.array(image_data)
text_as_int = np.array(text_as_int)

# Create training and validation splits
train_size = int(0.8 * len(image_data))
train_images, val_images = image_data[:train_size], image_data[train_size:]
train_labels, val_labels = text_as_int[:train_size], text_as_int[train_size:]


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed

def build_model(vocab_size, max_length):
    # Define the CNN for image feature extraction
    image_input = Input(shape=(image_height, image_width, num_channels))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)

    # Repeat the extracted features to match the sequence length
    x = tf.expand_dims(x, axis=1)
    x = tf.tile(x, [1, max_length, 1])

    # Define the RNN for sequence generation
    rnn_output = LSTM(256, return_sequences=True)(x)
    rnn_output = TimeDistributed(Dense(vocab_size, activation='softmax'))(rnn_output)

    # Define the model
    model = Model(inputs=image_input, outputs=rnn_output)
    return model

vocab_size = len(vocab)
model = build_model(vocab_size, max_length)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
# Train the model
EPOCHS = 20
history = model.fit(
    train_images, train_labels,
    epochs=EPOCHS,
    validation_data=(val_images, val_labels)
)


In [ ]:
# Function to generate text from image
def generate_text(model, image):
    prediction = model.predict(np.expand_dims(image, axis=0))
    predicted_text = ''.join([idx2char[np.argmax(char)] for char in prediction[0]])
    return predicted_text

# Example usage
example_image = val_images[0]  # Use an example image from the validation set
generated_text = generate_text(model, example_image)
print(generated_text)
